In [1]:
# E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one.
# Feel free to use either counting or a neural net.
# Evaluate the loss; Did it improve over a bigram model?

words = open('names.txt', 'r').read().splitlines()




['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [2]:
import torch

In [3]:
N = torch.zeros((27,27,27), dtype=torch.int32)


In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


In [5]:

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:],chs[2:]): 
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    N[ix1, ix2,ix3] += 1
 

In [6]:

# softmax creates better results, but the log prob becomes inf
# p = (N+1).float()
# softmax = torch.nn.Softmax(dim=-1)
# P = softmax(p)


# calculating probability with average of a 3d slice
P = (N+1).float()
P /= P.sum(2, keepdims=True)

g = torch.Generator().manual_seed(2899889987888)

P[21,6]


tensor([0.1739, 0.0435, 0.0217, 0.0217, 0.0217, 0.0652, 0.0435, 0.0217, 0.0217,
        0.1087, 0.0217, 0.0217, 0.0217, 0.0217, 0.0217, 0.0217, 0.0217, 0.0217,
        0.0217, 0.0217, 0.0435, 0.0435, 0.0217, 0.0217, 0.0217, 0.0652, 0.0217])

In [7]:


# Text generation loop

for i in range(5):
    out = []
    prev_chars = (0, 0)  # Initial pair of character indices
    
    # Text generation process
    while True:
        # Get the probability distribution for the next character given the previous two characters
        p = P[prev_chars[0], prev_chars[1]]
        
        # Sample the next character index based on the probability distribution
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        
        # Map the sampled index to its corresponding character
        next_char = itos[ix]
        
        # Append the next character to the output sequence
        out.append(next_char)
        
        # Update the previous characters for the next iteration
        prev_chars = (prev_chars[1], ix)
        
        # Break if the end-of-sequence token is encountered
        if next_char == '.':
            break
    
    # Print the generated text sequence
    print(''.join(out))


fain.
ibergbbi.
yus.
qah.
quiahrivedon.


In [8]:
 log_likelihood = 0.0
n = 0

for w in words:
#for w in ["andrejq"]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    prob = P[ix1, ix2,ix3]
  
        
    # Calculate the log probability
    logprob = torch.log(prob)
        
    # Add to log likelihood
    log_likelihood += logprob
    n += 1


    #print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-410414.9688)
nll=tensor(410414.9688)
2.092747449874878


In [44]:
# E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. 
#     Train the bigram and trigram models only on the training set.
#     Evaluate them on dev and test splits. What can you see?

# E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one.
# Feel free to use either counting or a neural net.
# Evaluate the loss; Did it improve over a bigram model?

words = open('names.txt', 'r').read().splitlines()




In [39]:
# create the training set of bigrams (x,y)
xs, ys = [], []

for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    print(ch1, ch2)
    xs.append(ix1)
    ys.append(ix2)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)

. e
e m
m m
m a
a .


In [40]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float()
xenc

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [41]:
xenc.shape

torch.Size([5, 27])

In [42]:
xenc.dtype

torch.float32

In [43]:
W = torch.randn((27, 1))
xenc @ W

tensor([[-0.8581],
        [ 0.8704],
        [ 0.4631],
        [ 0.4631],
        [ 0.4149]])